In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content
# Install mmcv
!pip install mmcv -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
# Clone mmcls repository
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/
# Install MMClassification from source
!pip install -e . 

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 554 kB 32.3 MB/s 
     |████████████████████████████████| 190 kB 62.3 MB/s 
  Created wheel for mmcv: filename=mmcv-1.6.0-py2.py3-none-any.whl size=848005 sha256=31770f5795d3a6b3d96ce08a179cbeb189b336d5d6eaef88a093e42ebd1d9c16
  Stored in directory: /root/.cache/pip/wheels/bc/d3/f7/5ad65ec39feb2546b12183f826ffe6bfbbd8a56aaa389111be
Successfully built mmcv
Cloning into 'mmclassification'...
remote: Enumerating objects: 8712, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 8712 (delta 0), reused 3 (delta 0), pack-reused 8701
Receiving objects: 100% (8712/8712), 9.87 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (6073/6073), done.
/content/mmclassification
Looking in indexes: https://pypi.o

In [3]:
# Check MMClassification installation
import torch, torchvision
import mmcls
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot
from mmcv import Config
from mmcls.apis import set_random_seed
import time
import os.path as osp
from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model
import gc
print(mmcls.__version__)

/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


0.23.1


In [4]:
def create_model(config_file,checkpoint_file,datasetpath,epochs=3,train_cfg=None):
  # Load the base config file
  cfg = Config.fromfile(config_file)

  # Modify the number of classes in the head.
  cfg.model.head.num_classes = 3

  # if custom train cfg defined ( for swin transformers we must define it)
  if train_cfg!=None:
    cfg.model.train_cfg=train_cfg

  cfg.model.head.topk = (1, )

  # Load the pre-trained model's checkpoint.
  cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')

  # Specify sample size and number of workers.
  cfg.data.samples_per_gpu = 8
  cfg.data.workers_per_gpu = 2

  # Specify the path and meta files of training dataset
  cfg.data.train.data_prefix = datasetpath+'/train_set/train_set'
  cfg.data.train.classes = datasetpath+'/clasess.txt'

  # Specify the path and meta files of validation dataset
  cfg.data.val.data_prefix = datasetpath+'/val_set/val_set'
  cfg.data.val.ann_file = datasetpath+'/val.txt'
  cfg.data.val.classes = datasetpath+'/clasess.txt'

  # Specify the path and meta files of test dataset
  cfg.data.test.data_prefix = datasetpath+'/test_set/test_set'
  cfg.data.test.ann_file = datasetpath+'/test.txt'
  cfg.data.test.classes = datasetpath+'/clasess.txt'


  # Modify the evaluation metric
  cfg.evaluation['metric_options']={'topk': (1, )}

  # Specify the optimizer
  cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
  cfg.optimizer_config = dict(grad_clip=None)

  # Specify the learning rate scheduler
  cfg.lr_config = dict(policy='step', step=1, gamma=0.1)
  cfg.runner = dict(type='EpochBasedRunner', max_epochs=epochs)

  # Specify the work directory
  cfg.work_dir = './work_dirs/Covid19-dataset'

  # Output logs for every 10 iterations
  cfg.log_config.interval = 10

  # Set the random seed and enable the deterministic option of cuDNN
  # to keep the results' reproducible.
  
  cfg.seed = 0
  set_random_seed(0, deterministic=True)

  cfg.gpu_ids = range(1)
  return cfg

In [5]:
def train(cfg):
  # Create the work directory
  mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
  # Build the classifier
  model = build_classifier(cfg.model)
  model.init_weights()
  # Build the dataset
  datasets = [build_dataset(cfg.data.train)]
  # Add `CLASSES` attributes to help visualization
  model.CLASSES = datasets[0].CLASSES
  # Start fine-tuning
  train_model(
      model,
      datasets,
      cfg,
      distributed=False,
      validate=True,
      timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
      meta=dict())
  gc.collect()


# Efficient-net

In [6]:
cfg_model_efficientnet=create_model(config_file='configs/efficientnet/efficientnet-b0_8xb32_in1k.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30
)
train(cfg=cfg_model_efficientnet)

2022-07-25 18:07:34,387 - mmcv - INFO - initialize EfficientNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth', 'prefix': 'backbone'}
2022-07-25 18:07:34,389 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

2022-07-25 18:07:36,613 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-07-25 18:07:36,619 - mmcv - INFO - 
backbone.layers.0.conv.weight - torch.Size([32, 3, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth 
 
2022-07-25 18:07:36,624 - mmcv - INFO - 
backbone.layers.0.bn.weight - torch.Size([32]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth 
 
2022-07-25 18:07:36,633 - mmcv - INFO - 
backbone.layers.0.bn.bias - torch.Size([32]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth 
 
2022-07-25 18:07:36,639 - mmcv - INFO - 
backbone.layers.1.0.depthwise_conv.conv.weight - torch.Size([32, 1, 3, 3]): 
PretrainedInit: load

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 2.8 task/s, elapsed: 23s, ETA:     0s

...

2022-07-25 18:15:14,442 - mmcls - INFO - Epoch(val) [28][9]	accuracy_top-1: 92.4242
2022-07-25 18:15:19,723 - mmcls - INFO - Epoch [29][10/32]	lr: 5.000e-31, eta: 0:00:22, time: 0.527, data_time: 0.418, memory: 949, loss: 0.5697
2022-07-25 18:15:22,843 - mmcls - INFO - Epoch [29][20/32]	lr: 5.000e-31, eta: 0:00:18, time: 0.312, data_time: 0.188, memory: 949, loss: 0.5731
2022-07-25 18:15:25,735 - mmcls - INFO - Epoch [29][30/32]	lr: 5.000e-31, eta: 0:00:13, time: 0.289, data_time: 0.187, memory: 949, loss: 0.4952
2022-07-25 18:15:25,870 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 39.9 task/s, elapsed: 2s, ETA:     0s

2022-07-25 18:15:27,686 - mmcls - INFO - Epoch(val) [29][9]	accuracy_top-1: 89.3939
2022-07-25 18:15:32,853 - mmcls - INFO - Epoch [30][10/32]	lr: 5.000e-32, eta: 0:00:09, time: 0.516, data_time: 0.410, memory: 949, loss: 0.5756
2022-07-25 18:15:35,675 - mmcls - INFO - Epoch [30][20/32]	lr: 5.000e-32, eta: 0:00:04, time: 0.283, data_time: 0.179, memory: 949, loss: 0.5574
2022-07-25 18:15:38,735 - mmcls - INFO - Epoch [30][30/32]	lr: 5.000e-32, eta: 0:00:00, time: 0.306, data_time: 0.207, memory: 949, loss: 0.6177
2022-07-25 18:15:39,154 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 40.8 task/s, elapsed: 2s, ETA:     0s

2022-07-25 18:15:40,938 - mmcls - INFO - Epoch(val) [30][9]	accuracy_top-1: 89.3939


# Vision transformer

In [7]:
cfg_model_vision=create_model(config_file='configs/vision_transformer/vit-base-p16_ft-64xb64_in1k-384.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30
)
train(cfg=cfg_model_vision)

2022-07-25 18:15:42,840 - mmcv - INFO - initialize VisionTransformer with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth', 'prefix': 'backbone'}
2022-07-25 18:15:42,842 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth" to /root/.cache/torch/hub/checkpoints/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth


  0%|          | 0.00/331M [00:00<?, ?B/s]

2022-07-25 18:16:15,789 - mmcv - INFO - initialize VisionTransformerClsHead with init_cfg {'type': 'Constant', 'layer': 'Linear', 'val': 0}
2022-07-25 18:16:15,791 - mmcv - INFO - 
backbone.cls_token - torch.Size([1, 1, 768]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth 
 
2022-07-25 18:16:15,794 - mmcv - INFO - 
backbone.pos_embed - torch.Size([1, 577, 768]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth 
 
2022-07-25 18:16:15,796 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([768, 3, 16, 16]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth 
 
2022-07-25 18:16:15,799 - mmcv - INFO - 
backbone.patch_embed.projectio

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 17.0 task/s, elapsed: 4s, ETA:     0s

...

2022-07-25 18:35:56,328 - mmcls - INFO - Epoch(val) [28][9]	accuracy_top-1: 30.3030
2022-07-25 18:36:08,846 - mmcls - INFO - Epoch [29][10/32]	lr: 5.000e-31, eta: 0:00:54, time: 1.218, data_time: 0.288, memory: 5533, loss: 1.6943
2022-07-25 18:36:18,648 - mmcls - INFO - Epoch [29][20/32]	lr: 5.000e-31, eta: 0:00:44, time: 0.980, data_time: 0.034, memory: 5533, loss: 1.9390
2022-07-25 18:36:28,672 - mmcls - INFO - Epoch [29][30/32]	lr: 5.000e-31, eta: 0:00:34, time: 1.002, data_time: 0.035, memory: 5533, loss: 1.8651
2022-07-25 18:36:29,925 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 18.1 task/s, elapsed: 4s, ETA:     0s

2022-07-25 18:36:38,096 - mmcls - INFO - Epoch(val) [29][9]	accuracy_top-1: 30.3030
2022-07-25 18:36:50,575 - mmcls - INFO - Epoch [30][10/32]	lr: 5.000e-32, eta: 0:00:22, time: 1.214, data_time: 0.288, memory: 5533, loss: 1.8251
2022-07-25 18:37:00,415 - mmcls - INFO - Epoch [30][20/32]	lr: 5.000e-32, eta: 0:00:12, time: 0.983, data_time: 0.034, memory: 5533, loss: 1.8312
2022-07-25 18:37:10,349 - mmcls - INFO - Epoch [30][30/32]	lr: 5.000e-32, eta: 0:00:02, time: 0.993, data_time: 0.034, memory: 5533, loss: 1.8524
2022-07-25 18:37:11,583 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 18.0 task/s, elapsed: 4s, ETA:     0s

2022-07-25 18:37:20,259 - mmcls - INFO - Epoch(val) [30][9]	accuracy_top-1: 30.3030


# Swin transformer

In [8]:
cfg_model_swin=create_model(config_file='configs/swin_transformer/swin-small_16xb64_in1k.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30,
                       train_cfg=dict(augments=[
                                      dict(type='BatchMixup', alpha=0.8, num_classes=3, prob=0.5),
                                      dict(type='BatchCutMix', alpha=1.0, num_classes=3, prob=0.5)
                                      ])
)
train(cfg=cfg_model_swin)

2022-07-25 18:37:22,056 - mmcv - INFO - initialize ImageClassifier with init_cfg [{'type': 'TruncNormal', 'layer': 'Linear', 'std': 0.02, 'bias': 0.0}, {'type': 'Constant', 'layer': 'LayerNorm', 'val': 1.0, 'bias': 0.0}]
2022-07-25 18:37:22,575 - mmcv - INFO - initialize SwinTransformer with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth', 'prefix': 'backbone'}
2022-07-25 18:37:22,577 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth" to /root/.cache/torch/hub/checkpoints/swin_small_patch4_window7_224-cc7a01c9.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth


  0%|          | 0.00/191M [00:00<?, ?B/s]

2022-07-25 18:37:45,286 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([96, 3, 4, 4]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth 
 
2022-07-25 18:37:45,288 - mmcv - INFO - 
backbone.patch_embed.projection.bias - torch.Size([96]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth 
 
2022-07-25 18:37:45,291 - mmcv - INFO - 
backbone.patch_embed.norm.weight - torch.Size([96]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth 
 
2022-07-25 18:37:45,297 - mmcv - INFO - 
backbone.patch_embed.norm.bias - torch.Size([96]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth 
 
2022-07-25 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 22.1 task/s, elapsed: 3s, ETA:     0s

2022-07-25 18:41:38,511 - mmcls - INFO - Epoch(val) [10][9]	accuracy_top-1: 43.9394
2022-07-25 18:41:46,589 - mmcls - INFO - Epoch [11][10/32]	lr: 5.000e-13, eta: 0:06:41, time: 0.806, data_time: 0.550, memory: 5533, loss: 1.0560
2022-07-25 18:41:51,985 - mmcls - INFO - Epoch [11][20/32]	lr: 5.000e-13, eta: 0:06:33, time: 0.540, data_time: 0.290, memory: 5533, loss: 1.0802
2022-07-25 18:41:58,231 - mmcls - INFO - Epoch [11][30/32]	lr: 5.000e-13, eta: 0:06:26, time: 0.625, data_time: 0.374, memory: 5533, loss: 1.0517
2022-07-25 18:41:58,767 - mmcls - INFO - Saving checkpoint at 11 epochs
2022-07-25 18:42:11,183 - mmcls - INFO - Epoch [12][10/32]	lr: 5.000e-14, eta: 0:06:21, time: 0.920, data_time: 0.671, memory: 5533, loss: 1.0476
2022-07-25 18:42:17,708 - mmcls - INFO - Epoch [12][20/32]	lr: 5.000e-14, eta: 0:06:15, time: 0.653, data_time: 0.379, memory: 5533, loss: 1.0719
2022-07-25 18:42:24,071 - mmcls - INFO - Epoch [12][30/32]	lr: 5.000e-14, eta: 0:06:09, time: 0.636, data_time: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 23.6 task/s, elapsed: 3s, ETA:     0s

2022-07-25 18:45:29,893 - mmcls - INFO - Epoch(val) [20][9]	accuracy_top-1: 43.9394
2022-07-25 18:45:38,101 - mmcls - INFO - Epoch [21][10/32]	lr: 5.000e-23, eta: 0:03:15, time: 0.819, data_time: 0.577, memory: 5533, loss: 1.0535
2022-07-25 18:45:42,926 - mmcls - INFO - Epoch [21][20/32]	lr: 5.000e-23, eta: 0:03:08, time: 0.482, data_time: 0.242, memory: 5533, loss: 1.0815
2022-07-25 18:45:49,334 - mmcls - INFO - Epoch [21][30/32]	lr: 5.000e-23, eta: 0:03:01, time: 0.641, data_time: 0.407, memory: 5533, loss: 1.0468
2022-07-25 18:45:50,185 - mmcls - INFO - Saving checkpoint at 21 epochs
2022-07-25 18:46:00,499 - mmcls - INFO - Epoch [22][10/32]	lr: 5.000e-24, eta: 0:02:54, time: 0.833, data_time: 0.570, memory: 5533, loss: 1.0539
2022-07-25 18:46:05,766 - mmcls - INFO - Epoch [22][20/32]	lr: 5.000e-24, eta: 0:02:48, time: 0.527, data_time: 0.270, memory: 5533, loss: 1.0974
2022-07-25 18:46:11,698 - mmcls - INFO - Epoch [22][30/32]	lr: 5.000e-24, eta: 0:02:41, time: 0.593, data_time: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 24.8 task/s, elapsed: 3s, ETA:     0s

2022-07-25 18:49:21,726 - mmcls - INFO - Epoch(val) [30][9]	accuracy_top-1: 43.9394
